## Cleaning

In [3]:
import pandas as pd
import numpy as np

In [4]:
raw = pd.read_csv("databases/raw.csv")

In [5]:
raw['name'].unique()

array(['Makrakomi, Greece', 'Lytton, BC', 'New York City, USA',
       'Port-au-prince, Haiti', 'Suva, Fiji', 'Texas, United States',
       'Bad Neuenahr-Ahrweiler, Germany', 'Verhoyansk, Russia',
       'Cizre, Turkey', 'Zhengzhou, China', 'Sicily, Italy',
       'Kupang, Indonesia', 'Madrid, Spain', 'Qaanaaq, Greenland',
       'New Orleans, USA'], dtype=object)

In [6]:
raw.dtypes 

name                 object
datetime             object
tempmax             float64
tempmin             float64
temp                float64
humidity            float64
precip              float64
precipcover         float64
snow                float64
snowdepth           float64
windgust            float64
windspeed           float64
sealevelpressure    float64
severerisk          float64
dtype: object

#### what percentage is NaN?

In [7]:
#cleaning: null value replacement
total = raw.isnull().sum().sort_values(ascending=False)

percent = (raw.isnull().sum()/raw.isnull().count()).sort_values(ascending=False)

missing_data = pd.concat([total,percent],axis=1, keys=['Total','Percent'])
missing_data

,Total,Percent
snow,279435,1.000000
severerisk,279435,1.000000
snowdepth,252172,0.902435
windgust,245343,0.877997
precip,87897,0.314553
sealevelpressure,38441,0.137567
humidity,23525,0.084188
windspeed,23421,0.083816
tempmax,23357,0.083587
tempmin,23357,0.083587


#### forward fill (one step) fill rest with zeros

In [8]:
raw = raw.fillna(method='ffill')
raw = raw.fillna(0)

#### from visual crossing weather dataset: concat on axis 1, drop columns with >90% missing data

In [9]:
#to concat the unique cities in axis 1 (right now they are in axis 0)
uniqueNames = raw.name.unique()

#create a data frame dictionary to store your data frames
df_dict = {elem : pd.DataFrame() for elem in uniqueNames}

for key in df_dict.keys():
    df_dict[key] = raw[:][raw.name == key]
    
df = pd.concat(df_dict.values(), axis=1)
df = df.apply(lambda x: pd.Series(x.dropna().values))
df.drop(columns=['name','snow','severerisk'],inplace=True)

In [10]:
df.columns

Index(['datetime', 'tempmax', 'tempmin', 'temp', 'humidity', 'precip',
       'precipcover', 'snowdepth', 'windgust', 'windspeed',
       ...
       'tempmax', 'tempmin', 'temp', 'humidity', 'precip', 'precipcover',
       'snowdepth', 'windgust', 'windspeed', 'sealevelpressure'],
      dtype='object', length=165)

#### save reconfigured dataset (rough)

In [11]:
df.to_csv('databases/cities_rough.csv', index = False)

#### import 'global avg temperature anomoly database from 1961-1990 average' as an extra feature (trend comparison)

In [12]:
world_temp = pd.read_csv("databases/temperature-anomaly.csv")

#### only use global temp from year 1970 forwards to concat with VC dataframe

In [14]:
world_temp = world_temp.loc[(world_temp['Entity'] == 'Global') & (world_temp['Year'] > 1970),['Year', 'Median temperature anomaly from 1961-1990 average']]
#shorten that name for easier use
world_temp.rename(columns={'Median temperature anomaly from 1961-1990 average': 'avg_temp_global'}, inplace=True)

In [21]:
#rough draft of cities, merge with world temp
df_rough= pd.read_csv('databases/cities_rough.csv')

#### merge dataframes

In [22]:
#in order to merge world temp and city temp (on year)
df_rough['datetime'] = df_rough['datetime'].astype('string')
world_temp['Year'] = world_temp['Year'].astype('string')

#only take year for merging purposes 
df_rough['year'] = df_rough['datetime'].str[:4]

In [23]:
df_rough = df_rough.merge(world_temp, how='left', left_on='year', right_on='Year')

In [24]:
#contains NA values, drop
df_rough.drop(columns=['Year'], inplace=True)
df_rough.drop(df_rough.tail(1).index,inplace=True)

In [25]:
#input missing values: temp for 2020 was 0.98, 2021 was 0.85
df_rough.loc[df_rough['year'] == '2020', 'avg_temp_global'] = 0.98
df_rough.loc[df_rough['year'] == '2021', 'avg_temp_global'] = 0.85

In [52]:
df_rough.to_csv('databases/full_df_rough.csv', index = False)

#### make index datetime for ARIMA time series

In [26]:
from datetime import datetime
# load data
def parse(x):
    return datetime.strptime(x, '%Y-%m-%d')

df_clean = pd.read_csv('databases/full_df_rough.csv',  parse_dates = ['datetime'], index_col=0, date_parser=parse)

df_clean.index.name = 'date'


In [27]:
df_clean.to_csv('databases/full_df_clean.csv', index=False)

In [28]:
#df_clean[df_clean.columns.drop(list(df_clean.filter(regex='14')))]


In [32]:
lytton = df_clean[['tempmax.1', 'tempmin.1', 'temp.1', 'humidity.1', 'precip.1', 'precipcover.1',
       'snowdepth.1', 'windgust.1', 'windspeed.1', 'sealevelpressure.1','avg_temp_global']]

In [33]:
lytton

,tempmax.1,tempmin.1,temp.1,humidity.1,precip.1,precipcover.1,snowdepth.1,windgust.1,windspeed.1,sealevelpressure.1,avg_temp_global
date,,,,,,,,,,,
1971-01-01,18.0,3.7,9.8,76.8,0.00,0.00,0.0,18.4,13.0,1020.1,-0.186
1971-01-02,18.0,3.7,9.8,76.8,0.00,0.00,0.0,18.4,13.0,1020.1,-0.186
1971-01-03,18.0,3.7,9.8,76.8,0.00,0.00,0.0,18.4,13.0,1020.1,-0.186
1971-01-04,18.0,3.7,9.8,76.8,0.00,0.00,0.0,18.4,13.0,1020.1,-0.186
1971-01-05,18.0,3.7,9.8,76.8,0.00,0.00,0.0,18.4,13.0,1020.1,-0.186
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,-20.0,-25.0,-22.1,62.3,0.00,0.00,27.0,33.5,19.5,1021.5,0.850
2021-12-28,-16.0,-22.0,-18.5,69.8,0.01,4.17,27.0,35.3,18.0,1019.3,0.850
2021-12-29,-15.0,-19.0,-17.2,67.9,0.00,0.00,27.0,35.3,22.0,1020.0,0.850


In [35]:
lytton.to_csv('databases/lytton.csv', index=True)